In [2]:
import os
import glob
import pandas as pd
drive.mount('/content/drive')
folder_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics"

all_files = glob.glob(os.path.join(folder_path, "*.csv"))

df_list = []

for file_path in all_files:

    df_temp = pd.read_csv(file_path)

    file_name = os.path.splitext(os.path.basename(file_path))[0]

    df_temp["source_file"] = file_name

    df_list.append(df_temp)

df_combined = pd.concat(df_list, ignore_index=True)

output_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data.csv"

df_combined.to_csv(output_path, index=False)
print(f"Combined data saved to: {output_path}")


NameError: name 'drive' is not defined

In [ ]:
print(folder_path)


/content/drive/MyDrive/Reddit_Data_Collection_statistics


In [ ]:
import pandas as pd

# Provide the full path to all_data.csv in Google Drive
csv_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data.csv"
df = pd.read_csv(csv_path)

# Filter rows by text length in the 4th column
df_filtered = df[(df.iloc[:, 4] > 10) & (df.iloc[:, 4] <= 250)]

# Save output
output_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_filtered.csv"
df_filtered.to_csv(output_path, index=False)
print(f"Filtered data saved to: {output_path}")


Filtered data saved to: /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_filtered.csv


In [ ]:
import pandas as pd
import math

# 1. Read the filtered data
input_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_filtered.csv"
df = pd.read_csv(input_path)

# 2. Shuffle the rows
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# 3. Split into multiple CSV files, each with up to 200 rows
max_rows = 200
total_rows = len(df_shuffled)

# Calculate how many chunks are needed
num_chunks = math.ceil(total_rows / max_rows)

for i in range(num_chunks):
    # Start and end index for the chunk
    start_index = i * max_rows
    end_index = min(start_index + max_rows, total_rows)

    # Slice the DataFrame
    chunk = df_shuffled.iloc[start_index:end_index]

    # Name the output file (e.g., part_1, part_2, ...)
    output_path = f"/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_{i+1}.csv"
    chunk.to_csv(output_path, index=False)
    print(f"Saved {output_path} with {len(chunk)} rows.")


Saved /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_1.csv with 200 rows.
Saved /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_2.csv with 200 rows.
Saved /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_3.csv with 200 rows.
Saved /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_4.csv with 200 rows.
Saved /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_5.csv with 200 rows.
Saved /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_6.csv with 200 rows.
Saved /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_7.csv with 200 rows.
Saved /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_8.csv with 200 rows.
Saved /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_9.csv with 200 rows.
Saved /content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_10.csv with 200 rows.
Saved /content/driv

In [ ]:
import pandas as pd
import requests

input_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_1.csv"
df = pd.read_csv(input_path)
# Define your prompts
prompt1 = """
Given a text, our task is to 1) finish a few diagnose of thought questions to analyze the thought patterns of a person. Then based on the diagnose of thought analysis, 2) identify if there is cognitive distortion in the speech; 3) Recognizing the specific types of the cognitive distortion. Here we consider the following common distortions:

All or Nothing Thinking/Polarized Thinking: I view a situation, a person or an event in “either-or” terms, fitting them into only two extreme categories instead of on a continuum.
Fortune telling (also called catastrophizing): I predict the future in negative terms and believe that what will happen will be so awful that I will not be able to stand it.
Emotional reasoning:  I believe my emotions reflect reality and let them guide my attitudes and judgments.
Labeling/Global Labeling: I put a fixed, global label, usually negative, on myself or others.
Mental Filter(): I pay attention to one or a few details and fail to see the whole picture.
Mind reading: I believe that I know the thoughts or intentions of others (or that they know my thoughts or intentions) without having sufficient evidence
Overgeneralization: I take isolated negative cases and generalize them, transforming them in a never-ending pattern, by repeatedly using words such as “always”, “never”, “ever”, “whole”, “entire”, etc
Personalization: I assume that others’ behaviors and external events concern (or are directed to) myself without considering other plausible explanations.
Should statements (also “musts”, “oughts”, “have tos”): I tell myself that events, people’s behaviors, and my own attitudes “should” be the way I expected them to be and not as they really are.
Blaming (others or oneself): I direct my attention to others as sources of my negative feelings and experiences, failing to consider my own responsibility; or, conversely, I take responsibility for others’ behaviors and attitudes.
What if?: I keep asking myself questions such as “what if something happens?”
Discounting the positive: I disqualify positive experiences or events insisting that they do not count.
Magnification/minimization: I evaluate myself, others, and situations placing greater importance on the negatives and/or placing much less importance on the positives.
Jumping to conclusions (also called arbitrary inference): I draw conclusions (negative or positive) from little or no confirmatory evidence.
Unfair comparisons: I compare myself with others who seem to do better than I do and place myself in a disadvantageous position.
"""

prompt2 = """
Based on this person's text, finish the following diagnosis of thought questions: 1. what is the situation? Find out the facts that are objective; what is this person thinking or imagining? Find out the thoughts or opinions that are subjective. 2. what makes this person think the thought is true or is not true? Find out the reasoning processes that support and do not support these thoughts. 3. why does this person come up with such reasoning process supporting the thought? What’s the underlying cognition mode of it?
"""

prompt3 = """
Please first answer: if there is cognitive distortion in the speech; Answer ’yes’ or ’no’; Please then answer: Recognizing the specific types of the cognitive distortion in the speech. There may be one type of cognitive distortion or multiple types involved. If there are multiple types, please give the top 2 dominant ones. Please only give the distortion type names separated by comma.
"""

# Function to send text to DeepSeek and get responses
def analyze_text_with_deepseek(text):
    # Replace with your DeepSeek API endpoint and API key
    api_url = "https://api.deepseek.com/v1/analyze"
    headers = {
        "Authorization": "",
        "Content-Type": "application/json"
    }

    # Combine prompts with the text
    full_prompt = f"{prompt1}\n\n{text}\n\n{prompt2}\n\n{prompt3}"

    # Send request to DeepSeek
    response = requests.post(api_url, headers=headers, json={"prompt": full_prompt})

    if response.status_code == 200:
        return response.json()  # Assuming DeepSeek returns a JSON response
    else:
        return {"error": f"API request failed with status code {response.status_code}"}

# Apply the function to each row in the CSV
df['Analysis'] = df.iloc[:, 3].apply(analyze_text_with_deepseek)

# Save the results to a new CSV file
df.to_csv('analyzed_data.csv', index=False)

print("Analysis complete. Results saved to 'analyzed_data.csv'.")

Analysis complete. Results saved to 'analyzed_data.csv'.


In [ ]:
import pandas as pd
from openai import OpenAI

# Initialize the OpenAI client with DeepSeek's API
client = OpenAI(api_key="", base_url="https://openrouter.ai/api/v1")

input_path = "/content/drive/MyDrive/Reddit_Data_Collection_statistics/all_data_part_2.csv"
df = pd.read_csv(input_path)

# Define your prompts
prompt1 = """
Given a text, our task is to 1) finish a few diagnose of thought questions to analyze the thought patterns of a person. Then based on the diagnose of thought analysis, 2) identify if there is cognitive distortion in the speech; 3) Recognizing the specific types of the cognitive distortion. Here we consider the following common distortions:

All or Nothing Thinking/Polarized Thinking: I view a situation, a person or an event in “either-or” terms, fitting them into only two extreme categories instead of on a continuum.
Fortune telling (also called catastrophizing): I predict the future in negative terms and believe that what will happen will be so awful that I will not be able to stand it.
Emotional reasoning:  I believe my emotions reflect reality and let them guide my attitudes and judgments.
Labeling/Global Labeling: I put a fixed, global label, usually negative, on myself or others.
Mental Filter(): I pay attention to one or a few details and fail to see the whole picture.
Mind reading: I believe that I know the thoughts or intentions of others (or that they know my thoughts or intentions) without having sufficient evidence
Overgeneralization: I take isolated negative cases and generalize them, transforming them in a never-ending pattern, by repeatedly using words such as “always”, “never”, “ever”, “whole”, “entire”, etc
Personalization: I assume that others’ behaviors and external events concern (or are directed to) myself without considering other plausible explanations.
Should statements (also “musts”, “oughts”, “have tos”): I tell myself that events, people’s behaviors, and my own attitudes “should” be the way I expected them to be and not as they really are.
Blaming (others or oneself): I direct my attention to others as sources of my negative feelings and experiences, failing to consider my own responsibility; or, conversely, I take responsibility for others’ behaviors and attitudes.
What if?: I keep asking myself questions such as “what if something happens?”
Discounting the positive: I disqualify positive experiences or events insisting that they do not count.
Magnification/minimization: I evaluate myself, others, and situations placing greater importance on the negatives and/or placing much less importance on the positives.
Jumping to conclusions (also called arbitrary inference): I draw conclusions (negative or positive) from little or no confirmatory evidence.
Unfair comparisons: I compare myself with others who seem to do better than I do and place myself in a disadvantageous position.
"""

prompt2 = """
Based on this person's text, finish the following diagnosis of thought questions: 1. what is the situation? Find out the facts that are objective; what is this person thinking or imagining? Find out the thoughts or opinions that are subjective. 2. what makes this person think the thought is true or is not true? Find out the reasoning processes that support and do not support these thoughts. 3. why does this person come up with such reasoning process supporting the thought? What’s the underlying cognition mode of it?
"""

prompt3 = """
Please first answer: if there is cognitive distortion in the speech; Answer ’yes’ or ’no’; Please then answer: Recognizing the specific types of the cognitive distortion in the speech. There may be one type of cognitive distortion or multiple types involved. If there are multiple types, please give the top 2 dominant ones. Please only give the distortion type names separated by comma.
"""

# Function to send text to DeepSeek and get responses
def analyze_text_with_deepseek(text):
    # Combine prompts with the text
    full_prompt = f"{prompt1}\n\n{text}\n\n{prompt2}\n\n{prompt3}"

    # Send request to DeepSeek
    response = client.chat.completions.create(
        model="deepseek/deepseek-r1",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": full_prompt},
        ],
        stream=False
    )

    # Extract the response content
    return response.choices[0].message.content

# Apply the function to the fourth column (index 3) in each row
df['Analysis'] = df.iloc[:, 3].apply(analyze_text_with_deepseek)

# Save the results to a new CSV file
df.to_csv('analyzed_data.csv', index=False)

print("Analysis complete. Results saved to 'analyzed_data.csv'.")

Analysis complete. Results saved to 'analyzed_data.csv'.


In [ ]:
import ollama
import pandas as pd

# Load the dataset
input_path = "~/Downloads/all_data_part_3.csv"  # Adjust path as needed
df = pd.read_csv(input_path)

# Define your prompts
prompt1 = """
Given a text, our task is to 1) finish a few diagnose of thought questions to analyze the thought patterns of a person. Then based on the diagnose of thought analysis, 2) identify if there is cognitive distortion in the speech; 3) Recognizing the specific types of the cognitive distortion. Here we consider the following common distortions:

All or Nothing Thinking/Polarized Thinking: I view a situation, a person or an event in “either-or” terms, fitting them into only two extreme categories instead of on a continuum.
Fortune telling (also called catastrophizing): I predict the future in negative terms and believe that what will happen will be so awful that I will not be able to stand it.
Emotional reasoning:  I believe my emotions reflect reality and let them guide my attitudes and judgments.
Labeling/Global Labeling: I put a fixed, global label, usually negative, on myself or others.
Mental Filter(): I pay attention to one or a fe
prompt2 = """
Based on this person's text, finish the following diagnosis of thought questions: 1. what is the situation? Find out the facts that are objective; what is this person thinking or imagining? Find out the thoughts or opinions that are subjective. 2. what makes this person think the thought is true or is not true? Find out the reasoning processes that support and do not support these thoughts. 3. why does this person come up with such reasoning process supporting the thought? What’s the underlying cognition mode of it?
"""

prompt3 = """Please first answer: if there is cognitive distortion in the speech; Answer ’yes’ or ’no’; Please then answer: Recognizing the specific types of the cognitive distortion in the speech. There may be one type of cognitive distortion or multiple types involved. If there are multiple types, please give the top 2 dominant ones. Please only give the distortion type names separated by comma.
"""

def analyze_text_with_ollama(text):
    full_prompt = f"{prompt1}\n\n{text}\n\n{prompt2}\n\n{prompt3}"
    
    response = ollama.chat(model="deepseek-r1:14b", messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": full_prompt}
    ])
    
    return response['message']['content']

# Apply function to dataset with progress tracking
total_rows = len(df)
for i, row in enumerate(df.iterrows(), start=1):
    text = row[1].iloc[3]  # Assuming text is in the fourth column
    df.at[row[0], 'Analysis'] = analyze_text_with_ollama(text)
    
    # Print progress
    print(f"Processed {i}/{total_rows} rows ({(i/total_rows)*100:.2f}%)", flush=True)

# Save results
df.to_csv('../data/analyzed_data.csv', index=False)
print("Analysis complete. Results saved to 'analyzed_data.csv'.")


SyntaxError: unterminated string literal (detected at line 18) (3417331284.py, line 18)

In [3]:
import ollama
import pandas as pd

# Load the dataset
input_path = "~/Downloads/all_data_Paper_based.csv"  # Adjust path as needed
df = pd.read_csv(input_path)

# Define your prompts
prompt1 = """
Given a text, our task is to 1) finish a few diagnose of thought questions to analyze the thought patterns of a person. Then based on the diagnose of thought analysis, 2) identify if there is cognitive distortion in the speech; 3) Recognizing the specific types of the cognitive distortion. Here we consider the following common distortions:

All or Nothing Thinking/Polarized Thinking: I view a situation, a person or an event in “either-or” terms, fitting them into only two extreme categories instead of on a continuum.
Fortune telling (also called catastrophizing): I predict the future in negative terms and believe that what will happen will be so awful that I will not be able to stand it.
Emotional reasoning:  I believe my emotions reflect reality and let them guide my attitudes and judgments.
Labeling/Global Labeling: I put a fixed, global label, usually negative, on myself or others.
Mental Filter(): I pay attention to one or a few details and fail to see the whole picture.
Mind reading: I believe that I know the thoughts or intentions of others (or that they know my thoughts or intentions) without having sufficient evidence
Overgeneralization: I take isolated negative cases and generalize them, transforming them in a endless pattern, by repeatedly using words such as “always”, “never”, “ever”, “whole”, “entire”, etc
Personalization: I assume that others’ behaviors and external events concern (or are directed to) myself without considering other plausible explanations.
Should statements (also “musts”, “oughts”, “have tos”): I tell myself that events, people’s behaviors, and my own attitudes “should” be the way I expected them to be and not as they really are.
Blaming (others or oneself): I direct my attention to others as sources of my negative feelings and experiences, failing to consider my own responsibility; or, conversely, I take responsibility for others’ behaviors and attitudes.
What if?: I keep asking myself questions such as “what if something happens?”
Discounting the positive: I disqualify positive experiences or events insisting that they do not count.
Magnification/minimization: I evaluate myself, others, and situations placing greater importance on the negatives and/or placing much less importance on the positives.
Jumping to conclusions (also called arbitrary inference): I draw conclusions (negative or positive) from little or no confirmatory evidence.
Unfair comparisons: I compare myself with others who seem to do better than I do and place myself in a disadvantageous position.
"""

prompt2 = """
Based on this person's text, finish the following diagnosis of thought questions: 1. what is the situation? Find out the facts that are objective; what is this person thinking or imagining? Find out the thoughts or opinions that are subjective. 2. what makes this person think the thought is true or is not true? Find out the reasoning processes that support and do not support these thoughts. 3. why does this person come up with such reasoning process supporting the thought? What’s the underlying cognition mode of it?
"""

prompt3 = """
Please first answer: if there is cognitive distortion in the speech; Answer ’yes’ or ’no’; Please then answer: Recognizing the specific types of the cognitive distortion in the speech. There may be one type of cognitive distortion or multiple types involved. If there are multiple types, please give the top 2 dominant ones. Please only give the distortion type names separated by comma.
"""

def analyze_text_with_ollama(text):
    full_prompt = f"{prompt1}\n\n{text}\n\n{prompt2}\n\n{prompt3}"
    
    response = ollama.chat(model="deepseek-r1:14b", messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": full_prompt}
    ])
    
    return response['message']['content']

# Apply function to dataset with progress tracking
total_rows = len(df)
for i, row in enumerate(df.iterrows(), start=1):
    text = row[1].iloc[3]  # Assuming text is in the fourth column
    df.at[row[0], 'Analysis'] = analyze_text_with_ollama(text)
    
    # Print progress
    print(f"Processed {i}/{total_rows} rows ({(i/total_rows)*100:.2f}%)", flush=True)

# Save results
df.to_csv('analyzed_data.csv', index=False)
print("Analysis complete. Results saved to 'analyzed_data.csv'.")

Processed 1/12 rows (8.33%)
Processed 2/12 rows (16.67%)
Processed 3/12 rows (25.00%)
Processed 4/12 rows (33.33%)
Processed 5/12 rows (41.67%)
Processed 6/12 rows (50.00%)
Processed 7/12 rows (58.33%)
Processed 8/12 rows (66.67%)
Processed 9/12 rows (75.00%)
Processed 10/12 rows (83.33%)
Processed 11/12 rows (91.67%)
Processed 12/12 rows (100.00%)
Analysis complete. Results saved to 'analyzed_data.csv'.


In [2]:
import ollama
import re
from collections import defaultdict
import pandas as pd

############################################
# 1) Define Your Three Prompts
############################################

prompt1 = """
Given a text, our task is to 1) finish a few diagnose of thought questions to analyze the thought patterns of a person. Then based on the diagnose of thought analysis, 2) identify if there is cognitive distortion in the speech; 3) Recognizing the specific types of the cognitive distortion. Here we consider the following common distortions:

All or Nothing Thinking/Polarized Thinking: I view a situation, a person or an event in “either-or” terms, fitting them into only two extreme categories instead of on a continuum.
Fortune telling (also called catastrophizing): I predict the future in negative terms and believe that what will happen will be so awful that I will not be able to stand it.
Emotional reasoning:  I believe my emotions reflect reality and let them guide my attitudes and judgments.
Labeling/Global Labeling: I put a fixed, global label, usually negative, on myself or others.
Mental Filter(): I pay attention to one or a few details and fail to see the whole picture.
Mind reading: I believe that I know the thoughts or intentions of others (or that they know my thoughts or intentions) without having sufficient evidence
Overgeneralization: I take isolated negative cases and generalize them, transforming them in a endless pattern, by repeatedly using words such as “always”, “never”, “ever”, “whole”, “entire”, etc
Personalization: I assume that others’ behaviors and external events concern (or are directed to) myself without considering other plausible explanations.
Should statements (also “musts”, “oughts”, “have tos”): I tell myself that events, people’s behaviors, and my own attitudes “should” be the way I expected them to be and not as they really are.
Blaming (others or oneself): I direct my attention to others as sources of my negative feelings and experiences, failing to consider my own responsibility; or, conversely, I take responsibility for others’ behaviors and attitudes.
What if?: I keep asking myself questions such as “what if something happens?”
Discounting the positive: I disqualify positive experiences or events insisting that they do not count.
Magnification/minimization: I evaluate myself, others, and situations placing greater importance on the negatives and/or placing much less importance on the positives.
Jumping to conclusions (also called arbitrary inference): I draw conclusions (negative or positive) from little or no confirmatory evidence.
Unfair comparisons: I compare myself with others who seem to do better than I do and place myself in a disadvantageous position.
"""

prompt2 = """
Based on this person's text, finish the following diagnosis of thought questions:
1. What is the situation? (objective facts)
2. What is the person thinking or imagining? (subjective opinions)
3. What makes the person think the thought is true or untrue? (reasoning)
4. Why does the person come up with such reasoning? (underlying cognition)
"""

prompt3 = """
Please first answer: if there is cognitive distortion in the speech; Answer 'yes' or 'no'; 
Please then answer: Recognizing the specific types of the cognitive distortion in the speech. 
There may be one type of cognitive distortion or multiple types involved. 
If there are multiple types, please give the top 2 dominant ones. 
Please only give the distortion type names separated by comma.
"""

############################################
# 2) Function to Call Ollama with the 3 Prompts
############################################

def analyze_text_with_ollama(text):
    """
    Calls your LLM with prompt1 + prompt2 + prompt3 and returns the entire LLM response (string).
    """
    full_prompt = f"{prompt1}\n\n{text}\n\n{prompt2}\n\n{prompt3}"
    
    response = ollama.chat(
        model="deepseek-r1:14b",  # or whichever model you are using
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": full_prompt}
        ]
    )
    return response['message']['content']

############################################
# 3) Parsing the LLM's Response
############################################

import re

def parse_distortions(llm_response: str):
    """
    Looks at the final lines of the LLM response to extract:
    1) "yes" or "no"
    2) If "yes," parse the rest of that line as comma-separated distortions.
    
    Returns a set of predicted distortions.
    If "no" or nothing found, returns an empty set.
    """
    lines = llm_response.strip().splitlines()
    distortions = set()
    
    # Search from the bottom up for a line containing "yes" or "no"
    for line in reversed(lines):
        line_stripped_lower = line.strip().lower()
        
        # If line has "yes" or "no"
        if "yes" in line_stripped_lower or "no" in line_stripped_lower:
            
            # If it says "no" and does not say "yes," then no distortion:
            if "no" in line_stripped_lower and "yes" not in line_stripped_lower:
                return set()
            
            # Otherwise, we assume there's "yes" plus possible distortions in this line
            # Remove up through the word "yes" (or "no"), plus punctuation after it
            # e.g. "Yes. The distortions are: X, Y" => remove "Yes."
            line_cleaned = re.sub(r"(?i)\b(?:yes|no)\b[\.\:\;\,\s]*", "", line).strip()
            
            # Remove trailing punctuation, e.g. final "." or "?" or "!"
            line_cleaned = re.sub(r"[.?!]+$", "", line_cleaned).strip()
            
            # Now split the remaining text by commas
            # e.g. "all or nothing thinking, mind reading"
            splitted = [chunk.strip() for chunk in line_cleaned.split(",") if chunk.strip()]
            
            # Add them to the set of distortions
            distortions.update(splitted)
            break  # Stop once we've found a line with yes/no
    
    return distortions


############################################
# 4) Ground Truth and Test Sentences
############################################

ground_truth = {
    "If I can't get a good mark, then I'm talentless.": "all_or_nothing_thinking",
    "My excitement will not allow me to perform on stage.": "fortune_telling",
    "My luck still allows me to hold this position.": "discounting_positive",
    "I feel that the event will be boring.": "emotional_reasoning",
    "My friend is short-sighted.": "labeling",
    "My mistake at the meeting suggests that I do not know how to behave in society.": "overgeneralization",
    "Bonus does not mean that I do a good job.": "magnification_minimization",
    "Despite the fact that I fulfilled the plan, my mistake in the report indicates that I am stupid.": "mental_filter",
    "He or she thinks I'm not attractive / handsome.": "mind_reading",
    "She / he talks to me rudely because I cannot explain my request.": "personalization",
    "I have to get married before twenty-five because that's the way it is.": "should_statements",
    "It is you who made me feel bad.": "blaming"
}

test_thoughts = list(ground_truth.keys())

############################################
# 5) Main: Evaluate Each Thought, Compute Metrics
############################################

def main():
    # We'll store everything in a small DataFrame for clarity
    results = []

    # We'll also accumulate confusion matrix stats for each distortion
    all_distortions = set(ground_truth.values())  # unique ground-truth distortions
    # For multi-distortion predictions, we consider it a "hit" if the ground-truth is in the predicted set

    # We track TP, FP, TN, FN for each known distortion
    tp = defaultdict(int)
    fp = defaultdict(int)
    tn = defaultdict(int)
    fn = defaultdict(int)

    for i, thought in enumerate(test_thoughts, start=1):
        print(f"===== Thought {i} / {len(test_thoughts)} =====")
        print(f"Text: {thought}\n")

        # 1) Call LLM
        llm_response = analyze_text_with_ollama(thought)
        print(f"LLM Response:\n{llm_response}\n")

        # 2) Parse Distortions
        predicted_set = parse_distortions(llm_response)
        if not predicted_set:
            predicted_str = "no_distortion"
        else:
            predicted_str = ", ".join(sorted(predicted_set))

        # 3) Ground Truth
        gt = ground_truth[thought].strip().lower()

        # 4) Metrics
        # For each distortion in all_distortions, update confusion matrix
        for dist in all_distortions:
            if dist == gt:
                # If this is the ground-truth distortion
                if dist in predicted_set:
                    tp[dist] += 1
                else:
                    fn[dist] += 1
            else:
                # Distortion is not the ground-truth
                if dist in predicted_set:
                    fp[dist] += 1
                else:
                    tn[dist] += 1

        # We'll also store the results in a table
        results.append({
            "Thought": thought,
            "GroundTruth": gt,
            "PredictedDistortions": predicted_str
        })

    # Summarize in a DataFrame
    df = pd.DataFrame(results)

    print("\n============================ RESULTS ============================\n")
    print(df)

    # ----------- Compute Precision, Recall, F1 -----------
    metrics_list = []
    for dist in sorted(all_distortions):
        d_tp = tp[dist]
        d_fp = fp[dist]
        d_tn = tn[dist]
        d_fn = fn[dist]

        precision = d_tp / (d_tp + d_fp) if (d_tp + d_fp) else 0
        recall = d_tp / (d_tp + d_fn) if (d_tp + d_fn) else 0
        f1 = (2*precision*recall) / (precision + recall) if (precision + recall) else 0

        metrics_list.append({
            "Distortion": dist,
            "TP": d_tp,
            "FP": d_fp,
            "TN": d_tn,
            "FN": d_fn,
            "Precision": round(precision, 3),
            "Recall": round(recall, 3),
            "F1 Score": round(f1, 3)
        })

    metrics_df = pd.DataFrame(metrics_list)
    print("\n================ DISTORTION METRICS ================\n")
    print(metrics_df)

    # ------- Overall Accuracy -------
    # "Correct" if the single ground-truth label is in the predicted set
    # (since each example has exactly one ground truth)
    correct_count = 0
    total_count = len(test_thoughts)
    for row in results:
        gt_label = row["GroundTruth"]
        pred_str = row["PredictedDistortions"].lower()
        
        pred_set = set(x.strip() for x in pred_str.split(",") if x.strip() and x != "no_distortion")
        if gt_label in pred_set:
            correct_count += 1

    accuracy = correct_count / total_count
    print(f"\nOverall accuracy: {accuracy:.3f}")

if __name__ == "__main__":
    main()


===== Thought 1 / 12 =====
Text: If I can't get a good mark, then I'm talentless.

LLM Response:
<think>
Okay, so I'm trying to figure out if there's a cognitive distortion in the statement: "If I can't get a good mark, then I'm talentless." Let me break it down step by step.

First, I need to understand what cognitive distortions are. From the list provided, they include things like all-or-nothing thinking, fortune telling, emotional reasoning, etc. So, I have to check which of these apply here.

Looking at the statement: The person is saying that if they can't get a good mark, it means they're talentless. That sounds pretty extreme—like there's no middle ground. If they don't achieve one good mark, their entire worth as talented is called into question. This seems like all-or-nothing thinking because it's framing the situation in extremes.

Also, maybe overgeneralization is involved here. The person is taking a single instance (not getting a good mark) and generalizing it to mean the

In [6]:
import ollama
import re
import json
from collections import defaultdict
import pandas as pd

############################################
# 1) Define Your Three Prompts
############################################

prompt1 = """
Given a text, our task is to 1) finish a few diagnose of thought questions to analyze the thought patterns of a person. Then based on the diagnose of thought analysis, 2) identify if there is cognitive distortion in the speech; 3) Recognizing the specific types of the cognitive distortion. Here we consider the following common distortions:

all_or_nothing_thinking: I view a situation, a person or an event in “either-or” terms, fitting them into only two extreme categories instead of on a continuum.
fortune_telling: (also called catastrophizing) I predict the future in negative terms and believe that what will happen will be so awful that I will not be able to stand it.
emotional_reasoning:  I believe my emotions reflect reality and let them guide my attitudes and judgments.
labeling: labeling/Global Labeling I put a fixed, global label, usually negative, on myself or others.
mental_filter: I pay attention to one or a few details and fail to see the whole picture.
mind_reading: I believe that I know the thoughts or intentions of others (or that they know my thoughts or intentions) without having sufficient evidence
overgeneralization: I take isolated negative cases and generalize them, transforming them in a endless pattern, by repeatedly using words such as “always”, “never”, “ever”, “whole”, “entire”, etc
personalization: I assume that others’ behaviors and external events concern (or are directed to) myself without considering other plausible explanations.
should_statements: (also “musts”, “oughts”, “have tos”) I tell myself that events, people’s behaviors, and my own attitudes “should” be the way I expected them to be and not as they really are.
blaming: (others or oneself) I direct my attention to others as sources of my negative feelings and experiences, failing to consider my own responsibility; or, conversely, I take responsibility for others’ behaviors and attitudes.
What_if?: I keep asking myself questions such as “what if something happens?”
discounting_the_positive: I disqualify positive experiences or events insisting that they do not count.
magnification/minimization: I evaluate myself, others, and situations placing greater importance on the negatives and/or placing much less importance on the positives.
jumping_to_conclusions: (also called arbitrary inference) I draw conclusions (negative or positive) from little or no confirmatory evidence.
unfair_comparisons: I compare myself with others who seem to do better than I do and place myself in a disadvantageous position.
"""

prompt2 = """
Based on this person's text, finish the following diagnosis of thought questions:
1. What is the situation? (objective facts)
2. What is the person thinking or imagining? (subjective opinions)
3. What makes the person think the thought is true or untrue? (reasoning)
4. Why does the person come up with such reasoning? (underlying cognition)
"""

#
# IMPORTANT: We now demand the answer in JSON only
#
prompt3 = """
Finally, provide your result *only* as valid JSON with no extra text or explanation.

The JSON must have this format:

{
  "distortion_exists": "yes" or "no",
  "distortion_types": [
     "Type1",
     "Type2"
  ]
}

Rules:
1. If you think there is no distortion, set "distortion_exists" to "no" and "distortion_types" to an empty list.
2. If you think there is distortion, set "distortion_exists" to "yes" and "distortion_types" to a list of *up to* two distortions.
3. No other keys or text should appear in your final response.
"""

############################################
# 2) Function to Call Ollama
############################################

import ollama
import re
import json

def extract_json_substring(raw_text: str) -> str:
    match = re.search(r"\{.*\}", raw_text, re.DOTALL)
    return match.group(0) if match else ""

def parse_json_distortions(raw_text: str):
    """
    - Extract JSON from raw LLM text
    - Attempt to parse
    - Return a Python dict or None if failure
    """
    json_str = extract_json_substring(raw_text)
    if not json_str:
        return None
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        return None

def analyze_text_with_ollama(text):
    system_instructions = """
    You are a helpful assistant.
    The user does not want to see your reasoning or chain-of-thought.
    You MUST provide only one valid JSON object as your final response. 
    No extra text, no code fences, no disclaimers, no tags like <think> or similar.

    If you need to reason, do it internally. 
    Only output the required JSON in the specified format:
    {
      "distortion_exists": "yes" or "no",
      "distortion_types": [ "Type1", "Type2" ]
    }
    Nothing else.
    """

    full_prompt = f"{prompt1}\n\n{text}\n\n{prompt2}\n\n{prompt3}"

    response = ollama.chat(
        model="deepseek-r1:14b",
        messages=[
            {"role": "system", "content": system_instructions},
            {"role": "user", "content": full_prompt}
        ]
    )

    # This is the raw text the model returns
    raw_text = response['message']['content']

    return raw_text

def parse_distortions(llm_response: str):
    data = parse_json_distortions(llm_response)
    if data is None:
        return set()

    if data.get("distortion_exists") == "no":
        return set()

    # Return a set of the strings in "distortion_types"
    return set(data.get("distortion_types", []))

############################################
# 4) Ground Truth and Test Sentences
############################################

ground_truth = {
    "If I can't get a good mark, then I'm talentless.": "all_or_nothing_thinking",
    "My excitement will not allow me to perform on stage.": "fortune_telling",
    "My luck still allows me to hold this position.": "discounting_positive",
    "I feel that the event will be boring.": "emotional_reasoning",
    "My friend is short-sighted.": "labeling",
    "My mistake at the meeting suggests that I do not know how to behave in society.": "overgeneralization",
    "Bonus does not mean that I do a good job.": "magnification_minimization",
    "Despite the fact that I fulfilled the plan, my mistake in the report indicates that I am stupid.": "mental_filter",
    "He or she thinks I'm not attractive / handsome.": "mind_reading",
    "She / he talks to me rudely because I cannot explain my request.": "personalization",
    "I have to get married before twenty-five because that's the way it is.": "should_statements",
    "It is you who made me feel bad.": "blaming"
}

test_thoughts = list(ground_truth.keys())

############################################
# 5) Main: Evaluate Each Thought, Compute Metrics
############################################

def main():
    # We'll store everything in a small DataFrame for clarity
    results = []

    # We'll also accumulate confusion matrix stats for each distortion
    all_distortions = set(ground_truth.values())  # unique ground-truth distortions
    # For multi-distortion predictions, we consider it a "hit" if the ground-truth is in the predicted set

    from collections import defaultdict
    tp = defaultdict(int)
    fp = defaultdict(int)
    tn = defaultdict(int)
    fn = defaultdict(int)

    for i, thought in enumerate(test_thoughts, start=1):
        print(f"===== Thought {i} / {len(test_thoughts)} =====")
        print(f"Text: {thought}\n")

        # 1) Call LLM
        llm_response = analyze_text_with_ollama(thought)
        print(f"Raw JSON Response:\n{llm_response}\n")

        # 2) Parse Distortions (JSON)
        predicted_set = parse_distortions(llm_response)
        if not predicted_set:
            predicted_str = "no_distortion"
        else:
            predicted_str = ", ".join(sorted(predicted_set))

        # 3) Ground Truth
        gt = ground_truth[thought].strip().lower()

        # 4) Update confusion matrix for each distortion
        for dist in all_distortions:
            if dist == gt:
                # If this is the ground-truth distortion
                if dist in predicted_set:
                    tp[dist] += 1
                else:
                    fn[dist] += 1
            else:
                # Distortion is not the ground-truth
                if dist in predicted_set:
                    fp[dist] += 1
                else:
                    tn[dist] += 1

        results.append({
            "Thought": thought,
            "GroundTruth": gt,
            "PredictedDistortions": predicted_str
        })

    # Summarize in a DataFrame
    df = pd.DataFrame(results)

    print("\n============================ RESULTS ============================\n")
    print(df)

    # ----------- Compute Precision, Recall, F1 -----------
    metrics_list = []
    for dist in sorted(all_distortions):
        d_tp = tp[dist]
        d_fp = fp[dist]
        d_tn = tn[dist]
        d_fn = fn[dist]

        precision = d_tp / (d_tp + d_fp) if (d_tp + d_fp) else 0
        recall = d_tp / (d_tp + d_fn) if (d_tp + d_fn) else 0
        f1 = (2*precision*recall) / (precision + recall) if (precision + recall) else 0

        metrics_list.append({
            "Distortion": dist,
            "TP": d_tp,
            "FP": d_fp,
            "TN": d_tn,
            "FN": d_fn,
            "Precision": round(precision, 3),
            "Recall": round(recall, 3),
            "F1 Score": round(f1, 3)
        })

    metrics_df = pd.DataFrame(metrics_list)
    print("\n================ DISTORTION METRICS ================\n")
    print(metrics_df)

    # ------- Overall Accuracy -------
    # "Correct" if the single ground-truth label is in the predicted set
    correct_count = 0
    total_count = len(test_thoughts)
    for row in results:
        gt_label = row["GroundTruth"]
        pred_str = row["PredictedDistortions"].lower()
        
        if gt_label in pred_str.split(", "):
            correct_count += 1

    accuracy = correct_count / total_count
    print(f"\nOverall accuracy: {accuracy:.3f}")

if __name__ == "__main__":
    main()


===== Thought 1 / 12 =====
Text: If I can't get a good mark, then I'm talentless.

Raw JSON Response:
<think>
Alright, let me try to figure out how to approach this problem. The user has given me a specific task involving cognitive distortions, which are errors in thinking that can lead to negative emotions and behaviors.

First, I need to understand the provided example to grasp what's expected. In the example, the statement is "If I can't get a good mark, then I'm talentless." The analysis identifies overgeneralization and labeling as the distortions. So, overgeneralizing because they assume that one instance (not getting a good mark) means they're talentless in general. Labeling is using a negative label about themselves.

Now, for my task, I need to apply this same process to another statement. However, in this case, since the user hasn't provided the specific text to analyze, I'm supposed to imagine what that might be. Let's assume the next text is something like, "If I fail this 

In [ ]:
import ollama
import re
import json
import time
from collections import defaultdict
import pandas as pd

############################################
# 1) Define Your Three Prompts
############################################

prompt1 = """
Given a text, our task is to 1) finish a few diagnose of thought questions to analyze the thought patterns of a person. Then based on the diagnose of thought analysis, 2) identify if there is cognitive distortion in the speech; 3) Recognizing the specific types of the cognitive distortion. Here we consider the following common distortions:

all_or_nothing_thinking: I view a situation, a person or an event in “either-or” terms, fitting them into only two extreme categories instead of on a continuum.
fortune_telling: (also called catastrophizing) I predict the future in negative terms and believe that what will happen will be so awful that I will not be able to stand it.
emotional_reasoning:  I believe my emotions reflect reality and let them guide my attitudes and judgments.
labeling: labeling/Global Labeling I put a fixed, global label, usually negative, on myself or others.
mental_filter: I pay attention to one or a few details and fail to see the whole picture.
mind_reading: I believe that I know the thoughts or intentions of others (or that they know my thoughts or intentions) without having sufficient evidence
overgeneralization: I take isolated negative cases and generalize them, transforming them in a endless pattern, by repeatedly using words such as “always”, “never”, “ever”, “whole”, “entire”, etc
personalization: I assume that others’ behaviors and external events concern (or are directed to) myself without considering other plausible explanations.
should_statements: (also “musts”, “oughts”, “have tos”) I tell myself that events, people’s behaviors, and my own attitudes “should” be the way I expected them to be and not as they really are.
blaming: (others or oneself) I direct my attention to others as sources of my negative feelings and experiences, failing to consider my own responsibility; or, conversely, I take responsibility for others’ behaviors and attitudes.
What_if?: I keep asking myself questions such as “what if something happens?”
discounting_the_positive: I disqualify positive experiences or events insisting that they do not count.
magnification/minimization: I evaluate myself, others, and situations placing greater importance on the negatives and/or placing much less importance on the positives.
jumping_to_conclusions: (also called arbitrary inference) I draw conclusions (negative or positive) from little or no confirmatory evidence.
unfair_comparisons: I compare myself with others who seem to do better than I do and place myself in a disadvantageous position.
"""

prompt2 = """
Based on this person's text, finish the following diagnosis of thought questions:
1. What is the situation? (objective facts)
2. What is the person thinking or imagining? (subjective opinions)
3. What makes the person think the thought is true or untrue? (reasoning)
4. Why does the person come up with such reasoning? (underlying cognition)
"""

#
# IMPORTANT: We now demand the answer in JSON only
#
prompt3 = """
Finally, provide your result *only* as valid JSON with no extra text or explanation.

The JSON must have this format:

{
  "distortion_exists": "yes" or "no",
  "distortion_types": [
     "Type1",
     "Type2"
  ]
}

Rules:
1. If you think there is no distortion, set "distortion_exists" to "no" and "distortion_types" to an empty list.
2. If you think there is distortion, set "distortion_exists" to "yes" and "distortion_types" to a list of *up to* two distortions.
3. No other keys or text should appear in your final response.
"""

############################################
# 2) Function to Call Ollama and JSON Extraction
############################################

def extract_json_substring(raw_text: str) -> str:
    match = re.search(r"\{.*\}", raw_text, re.DOTALL)
    return match.group(0) if match else ""

def parse_json_distortions(raw_text: str):
    """
    Extracts the JSON block from the raw text and returns it as a Python dict.
    """
    json_str = extract_json_substring(raw_text)
    if not json_str:
        return None
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        return None

def analyze_text_with_ollama(text):
    system_instructions = """
    You are a helpful assistant.
    The user does not want to see your reasoning or chain-of-thought.
    You MUST provide only one valid JSON object as your final response. 
    No extra text, no code fences, no disclaimers, no tags like <think> or similar.

    If you need to reason, do it internally. 
    Only output the required JSON in the specified format:
    {
      "distortion_exists": "yes" or "no",
      "distortion_types": [ "Type1", "Type2" ]
    }
    Nothing else.
    """

    full_prompt = f"{prompt1}\n\n{text}\n\n{prompt2}\n\n{prompt3}"

    response = ollama.chat(
        model="deepseek-r1:14b",
        messages=[
            {"role": "system", "content": system_instructions},
            {"role": "user", "content": full_prompt}
        ]
    )

    # Return the raw text response from the LLM
    return response['message']['content']

def parse_distortions(llm_response: str):
    data = parse_json_distortions(llm_response)
    if data is None:
        return set()

    if data.get("distortion_exists") == "no":
        return set()

    # Return a set of the strings in "distortion_types"
    return set(data.get("distortion_types", []))

############################################
# 3) Ground Truth and Test Sentences
############################################

ground_truth = {
    "If I can't get a good mark, then I'm talentless.": "all_or_nothing_thinking",
    "My excitement will not allow me to perform on stage.": "fortune_telling",
    "My luck still allows me to hold this position.": "discounting_the_positive",
    "I feel that the event will be boring.": "emotional_reasoning",
    "My friend is short-sighted.": "labeling",
    "My mistake at the meeting suggests that I do not know how to behave in society.": "overgeneralization",
    "Bonus does not mean that I do a good job.": "magnification/minimization",
    "Despite the fact that I fulfilled the plan, my mistake in the report indicates that I am stupid.": "mental_filter",
    "He or she thinks I'm not attractive / handsome.": "mind_reading",
    "She / he talks to me rudely because I cannot explain my request.": "personalization",
    "I have to get married before twenty-five because that's the way it is.": "should_statements",
    "It is you who made me feel bad.": "blaming"
}

test_thoughts = list(ground_truth.keys())

############################################
# 4) Main: Evaluate Each Thought, Compute Metrics, Consistency, and Latency
############################################

def main():
    # Prepare results and confusion matrix accumulators
    results = []
    all_distortions = set(ground_truth.values())  # unique ground-truth distortions

    tp = defaultdict(int)
    fp = defaultdict(int)
    tn = defaultdict(int)
    fn = defaultdict(int)

    overall_correct = 0
    total_examples = len(test_thoughts)
    
    overall_latency = 0.0
    overall_consistency = 0.0

    # Process each test thought
    for i, thought in enumerate(test_thoughts, start=1):
        print(f"===== Thought {i} / {len(test_thoughts)} =====")
        print(f"Text: {thought}\n")

        predicted_runs = []
        latencies = []

        # Run three times per thought
        for run in range(3):
            start_time = time.time()
            llm_response = analyze_text_with_ollama(thought)
            latency = time.time() - start_time
            latencies.append(latency)

            pred_set = parse_distortions(llm_response)
            predicted_runs.append(pred_set)
            print(f"Run {run+1}: Predicted: {pred_set} in {latency:.2f} sec")

        avg_latency = sum(latencies) / 3.0
        overall_latency += avg_latency

        # Compute consistency: use the frequency of identical predictions (using frozenset for hashability)
        prediction_counts = {}
        for pred in predicted_runs:
            key = frozenset(pred)
            prediction_counts[key] = prediction_counts.get(key, 0) + 1
        majority_count = max(prediction_counts.values())
        consistency = majority_count / 3.0
        overall_consistency += consistency

        # Determine final predicted set via majority vote.
        # In case of tie (all different), choose the first run.
        final_pred = None
        for pred in predicted_runs:
            if prediction_counts[frozenset(pred)] == majority_count:
                final_pred = pred
                break

        if not final_pred or len(final_pred) == 0:
            predicted_str = "no_distortion"
        else:
            predicted_str = ", ".join(sorted(final_pred))

        # Ground Truth
        gt = ground_truth[thought].strip().lower()

        # Update confusion matrix for each distortion
        for dist in all_distortions:
            if dist == gt:
                if dist in final_pred:
                    tp[dist] += 1
                else:
                    fn[dist] += 1
            else:
                if dist in final_pred:
                    fp[dist] += 1
                else:
                    tn[dist] += 1

        # Check if final prediction contains the ground truth label (for overall accuracy)
        if gt in final_pred:
            overall_correct += 1

        results.append({
            "Thought": thought,
            "GroundTruth": gt,
            "PredictedDistortions": predicted_str,
            "Consistency": round(consistency, 2),
            "Avg Latency (sec)": round(avg_latency, 2)
        })
        print("\n---------------------------------------------\n")

    # Summarize results in a DataFrame
    df = pd.DataFrame(results)
    print("\n============================ RESULTS ============================\n")
    print(df)

    # Compute precision, recall, and F1 metrics per distortion
    metrics_list = []
    for dist in sorted(all_distortions):
        d_tp = tp[dist]
        d_fp = fp[dist]
        d_tn = tn[dist]
        d_fn = fn[dist]

        precision = d_tp / (d_tp + d_fp) if (d_tp + d_fp) else 0
        recall = d_tp / (d_tp + d_fn) if (d_tp + d_fn) else 0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) else 0

        metrics_list.append({
            "Distortion": dist,
            "TP": d_tp,
            "FP": d_fp,
            "TN": d_tn,
            "FN": d_fn,
            "Precision": round(precision, 3),
            "Recall": round(recall, 3),
            "F1 Score": round(f1, 3)
        })

    metrics_df = pd.DataFrame(metrics_list)
    print("\n================ DISTORTION METRICS ================\n")
    print(metrics_df)

    # Overall accuracy based on final majority vote predictions
    accuracy = overall_correct / total_examples
    print(f"\nOverall accuracy: {accuracy:.3f}")

    # Report average latency and consistency across all examples
    avg_latency_overall = overall_latency / total_examples
    avg_consistency_overall = overall_consistency / total_examples
    print(f"\nAverage Latency per Thought (sec): {avg_latency_overall:.2f}")
    print(f"Average Consistency: {avg_consistency_overall:.2f}")

if __name__ == "__main__":
    main()


===== Thought 1 / 12 =====
Text: If I can't get a good mark, then I'm talentless.

Run 1: Predicted: {'overgeneralization', 'all_or_nothing_thinking'} in 11.95 sec
Run 2: Predicted: {'all_or_nothing_thinking', 'labeling'} in 7.89 sec
Run 3: Predicted: {'overgeneralization', 'all_or_nothing_thinking'} in 8.42 sec

---------------------------------------------

===== Thought 2 / 12 =====
Text: My excitement will not allow me to perform on stage.

Run 1: Predicted: {'emotional_reasoning', 'mental_filter'} in 10.28 sec
Run 2: Predicted: {'overgeneralization', 'fortune_telling'} in 6.71 sec
Run 3: Predicted: {'all_or_nothing_thinking', 'fortune_telling'} in 8.18 sec

---------------------------------------------

===== Thought 3 / 12 =====
Text: My luck still allows me to hold this position.

Run 1: Predicted: set() in 13.29 sec
Run 2: Predicted: {'emotional_reasoning', 'overgeneralization'} in 9.02 sec
Run 3: Predicted: {'overgeneralization'} in 8.31 sec

----------------------------------

In [ ]:
{
  "distortion_exists": "yes" or "no",
  "distortion_types": [
     "Type1",
     "Type2"
  ]
}

{'distortion_exists': 'yes', 'distortion_types': ['Type1', 'Type2']}